In [ ]:
!pip install matplotlib-venn

In [ ]:
!pip install isodate

In [ ]:
!apt-get -qq install -y libfluidsynth1

E: Package 'libfluidsynth1' has no installation candidate


In [ ]:
import requests
import pandas as pd

import isodate

API_KEY = "clès API"

SEARCH_URL = "https://www.googleapis.com/youtube/v3/search"
VIDEO_DETAILS_URL = "https://www.googleapis.com/youtube/v3/videos"
channel_id = "UCX6OQ3DkcsbYNE6H8uQQuVA"  # ID de la chaîne cible

video_ids = []
next_page_token = None


while True:
    search_params = {
        'part': 'snippet',
        'channelId': channel_id,
        'maxResults': 50,
        'order': 'date',
        'type': 'video',
        'pageToken': next_page_token,
        'key': API_KEY,
    }
    response = requests.get(SEARCH_URL, params=search_params)
    search_data = response.json()

    if 'items' not in search_data:
        print("Erreur dans la récupération des vidéos :", search_data)
        break

    video_ids.extend([item['id']['videoId'] for item in search_data['items']])

    next_page_token = search_data.get('nextPageToken')
    if not next_page_token:
        break

print(f"Nombre de vidéos récupérées : {len(video_ids)}")

video_details = []

def format_duration(iso_duration):
    """Formater la durée ISO 8601 en un format lisible (MM:SS)."""
    try:
        duration = isodate.parse_duration(iso_duration)
        total_seconds = int(duration.total_seconds())
        minutes, seconds = divmod(total_seconds, 60)
        return f"{minutes:02}:{seconds:02}"
    except:
        return "N/A"

for i in range(0, len(video_ids), 50):
    video_id_chunk = ",".join(video_ids[i:i+50])
    details_params = {
        'part': 'snippet,statistics,contentDetails',
        'id': video_id_chunk,
        'key': API_KEY,
    }
    details_response = requests.get(VIDEO_DETAILS_URL, params=details_params)
    details_data = details_response.json()

    if 'items' not in details_data:
        print("Erreur dans la récupération des détails :", details_data)
        continue

    for video in details_data['items']:
        snippet = video['snippet']
        statistics = video.get('statistics', {})
        content_details = video.get('contentDetails', {})

        video_details.append({
            'Video ID': video['id'],
            'Title': snippet['title'],
            'Description': snippet['description'],
            'Published Date': snippet['publishedAt'],
            'Tags': ", ".join(snippet.get('tags', [])),
            'Views': statistics.get('viewCount', 'N/A'),
            'Likes': statistics.get('likeCount', 'N/A'),
            'Comments': statistics.get('commentCount', 'N/A'),
            'Duration': format_duration(content_details.get('duration', 'N/A')),
            'Product Placement': content_details.get('hasCustomThumbnail', False)
        })

print(f"Nombre de vidéos avec détails récupérées : {len(video_details)}")

if video_details:
    df = pd.DataFrame(video_details)
    print("Aperçu des données :", df.head())
    df.to_csv("mrbeast_videos_with_placements.csv", index=False)
    print("Les données ont été exportées dans 'mrbeast_videos_with_placements.csv'.")
else:
    print("Aucune donnée à exporter.")



Nombre de vidéos récupérées : 502
Nombre de vidéos avec détails récupérées : 502
Aperçu des données :       Video ID                                   Title  \
0  DBPYB8WkkDc  Bring Me Back Snow From The North Pole   
1  zvBXpoBegwE       I Got T-Series To Subscribe To Me   
2  LJDN7dzrErg                 Never Doing This Again…   
3  SWc8fI_9iqA          Ronaldo Teaches Me How To SIUU   
4  0BjlBnfHcHM            Beat Ronaldo, Win $1,000,000   

                                         Description        Published Date  \
0                                                #ad  2024-12-10T17:00:00Z   
1                                                     2024-12-09T17:00:01Z   
2                                                     2024-12-03T17:30:00Z   
3                                                     2024-12-01T17:00:00Z   
4  I still can’t believe we got all of these athl...  2024-11-30T17:00:00Z   

  Tags      Views    Likes Comments Duration  Product Placement  
0         7581

In [ ]:
df_mrbeast = pd.read_csv("/content/mrbeast_videos_with_placements.csv")
df_mrbeast.head()

,Video ID,Title,Description,Published Date,Tags,Views,Likes,Comments,Duration,Product Placement
0,DBPYB8WkkDc,Bring Me Back Snow From The North Pole,#ad,2024-12-10T17:00:00Z,NaN,7581934,731397,1366.0,00:55,False
1,zvBXpoBegwE,I Got T-Series To Subscribe To Me,NaN,2024-12-09T17:00:01Z,NaN,12425377,1100302,3713.0,00:31,False
2,LJDN7dzrErg,Never Doing This Again…,NaN,2024-12-03T17:30:00Z,NaN,17340568,995511,3293.0,00:08,False
3,SWc8fI_9iqA,Ronaldo Teaches Me How To SIUU,NaN,2024-12-01T17:00:00Z,NaN,78386966,4159261,16509.0,00:15,False
4,0BjlBnfHcHM,"Beat Ronaldo, Win $1,000,000",I still can’t believe we got all of these athl...,2024-11-30T17:00:00Z,NaN,141061363,6295361,95294.0,22:45,False


In [ ]:
df_mrbeast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502 entries, 0 to 501
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Video ID           502 non-null    object 
 1   Title              502 non-null    object 
 2   Description        447 non-null    object 
 3   Published Date     502 non-null    object 
 4   Tags               232 non-null    object 
 5   Views              502 non-null    int64  
 6   Likes              502 non-null    int64  
 7   Comments           501 non-null    float64
 8   Duration           502 non-null    object 
 9   Product Placement  502 non-null    bool   
dtypes: bool(1), float64(1), int64(2), object(6)
memory usage: 35.9+ KB


In [ ]:
df_mrbeast['Published Date'] = pd.to_datetime(df_mrbeast['Published Date'])

In [ ]:
df_mrbeast = df_mrbeast.sort_values(by='Published Date', ascending=False)

In [ ]:
# classer par durée de video
df_mrbeast = df_mrbeast.sort_values(by='Duration', ascending=True)
df_mrbeast

,Video ID,Title,Description,Published Date,Tags,Views,Likes,Comments,Duration,Product Placement
2,LJDN7dzrErg,Never Doing This Again…,NaN,2024-12-03T17:30:00Z,NaN,17340568,995511,3293.0,00:08,False
30,YyNz4--6fro,World’s Smartest Dog,NaN,2024-05-16T16:00:00Z,NaN,185721626,8746569,31545.0,00:12,False
67,SHOV58tdNiI,Miranda Cosgrove Said What?,NaN,2023-10-15T17:00:00Z,NaN,233144358,11645182,59725.0,00:15,False
3,SWc8fI_9iqA,Ronaldo Teaches Me How To SIUU,NaN,2024-12-01T17:00:00Z,NaN,78386966,4159261,16509.0,00:15,False
8,9UtcHPCEBgg,1 Subscriber = 1 Penny,NaN,2024-10-19T18:00:00Z,NaN,166455267,10398770,10844.0,00:17,False
...,...,...,...,...,...,...,...,...,...,...
13,snX5YyflrGw,"100 Identical Twins Fight For $250,000",I’ve never seen this many twins in one place b...,2024-09-28T16:00:00Z,NaN,116807928,2976316,25732.0,35:40,False
39,l-nMKJ5J3Uc,"Ages 1 - 100 Decide Who Wins $250,000",I can’t believe who they picked\nThanks Top Tr...,2024-04-20T15:59:59Z,NaN,238926248,5818755,208326.0,40:02,False
23,4SNThp0YiU4,"50 YouTubers Fight For $1,000,000",I can’t believe how intense this video got\nTh...,2024-07-13T16:00:01Z,NaN,291391403,9000501,344515.0,41:27,False
326,AS5CxLCWq-Q,Watching Dance Till You're Dead For 10 Hours,I WATCHED THE WHOLE VIDEO\nEnter $500 Giveaway...,2017-03-21T22:40:12Z,"dance, till, youre, dead, remix, meme, vine, b...",54367650,2625918,NaN,600:01,False


In [ ]:
import requests
import pandas as pd
import isodate

API_KEY = "API KEY"

SEARCH_URL = "https://www.googleapis.com/youtube/v3/search"
VIDEO_DETAILS_URL = "https://www.googleapis.com/youtube/v3/videos"
channel_id = "UCX6OQ3DkcsbYNE6H8uQQuVA"
video_ids = []
next_page_token = None

while True:
    search_params = {
        'part': 'snippet',
        'channelId': channel_id,
        'maxResults': 50,
        'order': 'date',
        'type': 'video',
        'pageToken': next_page_token,
        'key': API_KEY,
    }
    response = requests.get(SEARCH_URL, params=search_params)
    search_data = response.json()

    if 'items' not in search_data:
        print("Erreur dans la récupération des vidéos :", search_data)
        break

    video_ids.extend([item['id']['videoId'] for item in search_data['items']])

    next_page_token = search_data.get('nextPageToken')
    if not next_page_token:
        break

print(f"Nombre de vidéos récupérées : {len(video_ids)}")

video_details = []

def format_duration(iso_duration):
    """Formater la durée ISO 8601 en un format lisible (MM:SS)."""
    try:
        duration = isodate.parse_duration(iso_duration)
        total_seconds = int(duration.total_seconds())
        minutes, seconds = divmod(total_seconds, 60)
        return f"{minutes:02}:{seconds:02}", total_seconds
    except:
        return "N/A", 0

for i in range(0, len(video_ids), 50):
    video_id_chunk = ",".join(video_ids[i:i+50])
    details_params = {
        'part': 'snippet,statistics,contentDetails',
        'id': video_id_chunk,
        'key': API_KEY,
    }
    details_response = requests.get(VIDEO_DETAILS_URL, params=details_params)
    details_data = details_response.json()

    if 'items' not in details_data:
        print("Erreur dans la récupération des détails :", details_data)
        continue

    for video in details_data['items']:
        snippet = video['snippet']
        statistics = video.get('statistics', {})
        content_details = video.get('contentDetails', {})

        duration_formatted, total_seconds = format_duration(content_details.get('duration', 'N/A'))
        is_short = total_seconds <= 60

        video_details.append({
            'Video ID': video['id'],
            'Title': snippet['title'],
            'Description': snippet['description'],
            'Published Date': snippet['publishedAt'],
            'Tags': ", ".join(snippet.get('tags', [])),
            'Views': statistics.get('viewCount', 'N/A'),
            'Likes': statistics.get('likeCount', 'N/A'),
            'Comments': statistics.get('commentCount', 'N/A'),
            'Duration': duration_formatted,
            'Is Short': is_short,
            'Product Placement': content_details.get('hasCustomThumbnail', False),
        })

print(f"Nombre de vidéos avec détails récupérées : {len(video_details)}")


if video_details:
    df = pd.DataFrame(video_details)
    print("Aperçu des données :", df.head())
    df.to_csv("mrbeast_videos_with_shorts.csv", index=False)
    print("Les données ont été exportées dans 'mrbeast_videos_with_shorts.csv'.")
else:
    print("Aucune donnée à exporter.")


Nombre de vidéos récupérées : 502
Nombre de vidéos avec détails récupérées : 502
Aperçu des données :       Video ID                                   Title  \
0  DBPYB8WkkDc  Bring Me Back Snow From The North Pole   
1  zvBXpoBegwE       I Got T-Series To Subscribe To Me   
2  LJDN7dzrErg                 Never Doing This Again…   
3  SWc8fI_9iqA          Ronaldo Teaches Me How To SIUU   
4  0BjlBnfHcHM            Beat Ronaldo, Win $1,000,000   

                                         Description        Published Date  \
0                                                #ad  2024-12-10T17:00:00Z   
1                                                     2024-12-09T17:00:01Z   
2                                                     2024-12-03T17:30:00Z   
3                                                     2024-12-01T17:00:00Z   
4  I still can’t believe we got all of these athl...  2024-11-30T17:00:00Z   

  Tags      Views    Likes Comments Duration  Is Short  Product Placement  
0   

In [ ]:

df_mrbeast = pd.read_csv("/content/mrbeast_videos_with_shorts.csv")
df_mrbeast.head()

NameError: name 'pd' is not defined

In [ ]:
df_mrbeast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502 entries, 0 to 501
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Video ID           502 non-null    object 
 1   Title              502 non-null    object 
 2   Description        447 non-null    object 
 3   Published Date     502 non-null    object 
 4   Tags               232 non-null    object 
 5   Views              502 non-null    int64  
 6   Likes              502 non-null    int64  
 7   Comments           501 non-null    float64
 8   Duration           502 non-null    object 
 9   Is Short           502 non-null    bool   
 10  Product Placement  502 non-null    bool   
dtypes: bool(2), float64(1), int64(2), object(6)
memory usage: 36.4+ KB


In [ ]:
df_mrbeast['Published Date'] = pd.to_datetime(df_mrbeast['Published Date'])

In [ ]:
df_mrbeast.isnull().sum()

,0
Video ID,0
Title,0
Description,0
Published Date,0
Tags,270
Views,0
Likes,0
Comments,1
Duration,0
Is Short,0


In [ ]:
df_mrbeast['Description'].fillna('', inplace=True)

In [ ]:
df_mrbeast.isnull().sum()

,0
Video ID,0
Title,0
Description,0
Published Date,0
Tags,0
Views,0
Likes,0
Comments,1
Duration,0
Is Short,0


In [ ]:
df_mrbeast = df_mrbeast.drop(columns="Description")

In [ ]:
import csv

input_file = "videos_mrbeast.csv"
output_file = "cleaned_file.csv"

with open(input_file, "r", encoding="utf-8", errors="ignore") as infile, open(output_file, "w", encoding="utf-8", newline="") as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    for row in reader:
        new_row = []
        for field in row:
            if field.count('"') % 2 != 0:
                field = field.replace('"', '""')
            new_row.append(field)
        writer.writerow(new_row)


In [ ]:
df_mrbeast.to_csv('videos_mrbeast.csv', index=False)

In [ ]:
df_top20 = pd.read_csv("/content/top_20_enriched.csv")
df_top20.head()

,Title,Country_x,Subscribers_x,Video Views,Uploads_x,Channel Link,Channel Type,Created Date,Subscriber Rank,Video View Rank,Country Rank,Estimated Monthly Income,Subscribers Last 30 Days,Views Last 30 Days,Yearly Income,Daily Subscribers Average,Daily Views Average
0,mrbeast,US,336000000,66485783433,836,https://youtube.com/@mrbeast,entertainment,"Feb 20th, 2012",1st,13th,1st,€760.4K - €12.2M,6M,3.342B,€9.1M - €146M,+200K,"+111,415,937"
1,vlad and niki,US,129000000,98963573244,790,https://youtube.com/@vladandniki,entertainment,"Apr 23rd, 2018",6th,7th,3rd,€344.4K - €5.5M,2M,1.514B,€4.1M - €66.1M,+66.6K,"+50,458,586"
2,✿ kids diana show,US,128000000,108833430096,1280,https://youtube.com/@kidsdianashow,entertainment,"May 12th, 2015",7th,5th,4th,€326.5K - €5.2M,1M,1.435B,€3.9M - €62.7M,+33.3K,"+47,832,263"
3,like nastya,US,123000000,107885644820,916,https://youtube.com/@likenastyaofficial,entertainment,"Dec 6th, 2016",8th,6th,5th,€335.9K - €5.4M,1M,1.476B,€4M - €64.5M,+33.3K,"+49,212,736"
4,pewdiepie,JP,110000000,29452005379,4800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_top20.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Title                      20 non-null     object
 1   Country_x                  18 non-null     object
 2   Subscribers_x              20 non-null     int64 
 3   Video Views                20 non-null     int64 
 4   Uploads_x                  20 non-null     int64 
 5   Channel Link               10 non-null     object
 6   Channel Type               10 non-null     object
 7   Created Date               10 non-null     object
 8   Subscriber Rank            10 non-null     object
 9   Video View Rank            10 non-null     object
 10  Country Rank               10 non-null     object
 11  Estimated Monthly Income   10 non-null     object
 12  Subscribers Last 30 Days   10 non-null     object
 13  Views Last 30 Days         10 non-null     object
 14  Yearly Incom

In [ ]:
df_top20.shape

(20, 17)

In [ ]:
df_top20.isnull().sum()

,0
Title,0
Country_x,2
Subscribers_x,0
Video Views,0
Uploads_x,0
Channel Link,10
Channel Type,10
Created Date,10
Subscriber Rank,10
Video View Rank,10


In [ ]:
#passer les ranks en intenger
df_top20['Subscriber Rank'] = pd.to_numeric(df_top20['Subscriber Rank'], errors='coerce')
df_top20 = df_top20.dropna(subset=['Subscriber Rank']) # Drop rows with NaN in 'Subscriber Rank'
df_top20['Subscriber Rank'] = df_top20['Subscriber Rank'].astype(int)

In [ ]:
df_top20

,Title,Country_x,Subscribers_x,Video Views,Uploads_x,Channel Link,Channel Type,Created Date,Subscriber Rank,Video View Rank,Country Rank,Estimated Monthly Income,Subscribers Last 30 Days,Views Last 30 Days,Yearly Income,Daily Subscribers Average,Daily Views Average
